<a href="https://colab.research.google.com/github/khshohelrana/Thesis_Purpose_Model/blob/main/betray's_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Function to load videos (grayscale) with 35 FPS and max 70 frames
def load_videos(video_dir, frame_size=(720, 720), max_frames=70, fps=35):
    video_data = []
    labels = []
    for label_dir in os.listdir(video_dir):
        label_path = os.path.join(video_dir, label_dir)
        if os.path.isdir(label_path):
            label = label_dir
            for video_file in os.listdir(label_path):
                video_path = os.path.join(label_path, video_file)
                frames = []
                cap = cv2.VideoCapture(video_path)
                frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Original frame rate of video
                frame_interval = int(frame_rate // fps) if frame_rate > fps else 1

                while len(frames) < max_frames:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % frame_interval == 0:
                        frame = cv2.resize(frame, frame_size)
                        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                        frames.append(frame_gray)
                cap.release()

                # If the video has fewer frames, pad with zeros at the end
                while len(frames) < max_frames:
                    frames.append(np.zeros(frame_size, dtype=np.uint8))

                video_data.append(np.array(frames))
                labels.append(label)
    return np.array(video_data), np.array(labels)

# Load videos
video_dir = '/content/drive/MyDrive/Best'  # Replace with your video directory path
X, y = load_videos(video_dir, frame_size=(720, 720), max_frames=70, fps=35)

# Preprocess labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Normalize pixel values
X = X / 255.0

# Reshape X for PyTorch: [samples, channels, height, width]
X = X.reshape(X.shape[0], 70, X.shape[2], X.shape[3])  # Channels = max_frames = 70

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 3D CNN model
class CNN3D(nn.Module):
    def __init__(self, num_classes):
        super(CNN3D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=70, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv1_bn = nn.BatchNorm2d(256)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2_bn = nn.BatchNorm2d(512)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self._get_flattened_size(), 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def _get_flattened_size(self):
        dummy_input = torch.rand(1, 70, 720, 720)
        x = self.pool1(F.relu(self.conv1_bn(self.conv1(dummy_input))))
        x = self.pool2(F.relu(self.conv2_bn(self.conv2(x))))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool2(F.relu(self.conv2_bn(self.conv2(x))))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize model, optimizer, and loss function
num_classes = len(np.unique(y_train))
model = CNN3D(num_classes)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
train_dataset = list(zip(X_train_torch, y_train_torch))
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.long)
test_dataset = list(zip(X_test_torch, y_test_torch))
val_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Training loop
num_epochs = 18
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        correct_train += (outputs.argmax(1) == labels).sum().item()
        total_train += labels.size(0)
    train_loss = running_loss / len(train_loader)
    train_acc = correct_train / total_train
    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            correct_val += (outputs.argmax(1) == labels).sum().item()
            total_val += labels.size(0)
    val_loss /= len(val_loader)
    val_acc = correct_val / total_val
    # Store metrics
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

# Plot training and validation curves
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

# Evaluate on test set and compute confusion matrix
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        preds = outputs.argmax(1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n")
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))